In [21]:
import pandas as pd
import numpy as np

games_df = pd.read_csv('../data/src/games.csv')
boxscore_df = pd.read_csv('../data/src/boxscore.csv')
boxscore_df.head()

,game_id,team_name,player_name,MP,FG,FGA,3P,3PA,FT,FTA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,is_starter
0,36a784f4-225e-4a4d-a2b5-431dd5492874,Sacramento Kings,Corliss Williamson,37:20,7.0,11.0,0.0,0.0,0.0,0.0,...,3.0,4.0,4.0,1.0,1.0,4.0,5.0,14.0,-2.0,1
1,36a784f4-225e-4a4d-a2b5-431dd5492874,Sacramento Kings,Mitch Richmond,32:00,6.0,12.0,1.0,4.0,1.0,1.0,...,5.0,5.0,3.0,1.0,2.0,3.0,1.0,14.0,-12.0,1
2,36a784f4-225e-4a4d-a2b5-431dd5492874,Sacramento Kings,Olden Polynice,31:34,0.0,4.0,0.0,0.0,1.0,4.0,...,5.0,7.0,3.0,1.0,0.0,4.0,3.0,1.0,-12.0,1
3,36a784f4-225e-4a4d-a2b5-431dd5492874,Sacramento Kings,Mahmoud Abdul-Rauf,29:27,7.0,13.0,1.0,2.0,2.0,2.0,...,2.0,2.0,5.0,1.0,1.0,2.0,2.0,17.0,-7.0,1
4,36a784f4-225e-4a4d-a2b5-431dd5492874,Sacramento Kings,Brian Grant,25:13,3.0,11.0,0.0,0.0,2.0,2.0,...,5.0,6.0,0.0,0.0,2.0,1.0,2.0,8.0,-7.0,1


In [22]:
duplicate_game_ids_games = games_df[games_df.duplicated(subset='game_id', keep=False)]
print(duplicate_game_ids_games)


       season_start_year              away_team  points_away  \
73                  1996        New York Knicks          101   
74                  1996           Phoenix Suns           79   
75                  1996    Seattle SuperSonics          110   
76                  1996         Denver Nuggets          104   
77                  1996  Golden State Warriors          102   
...                  ...                    ...          ...   
34051               2022             Miami Heat           93   
34052               2022             Miami Heat          111   
34053               2022         Denver Nuggets          109   
34054               2022         Denver Nuggets          108   
34055               2022             Miami Heat           89   

                 home_team  points_home  attendance notes start_ET  \
73     Vancouver Grizzlies           82     17356.0   NaN      NaN   
74           Chicago Bulls           97     23807.0   NaN      NaN   
75        Sacramento 

In [17]:
mask = boxscore_df.isin(['Player Suspended', 'Did Not Play', 'Did Not Dress', 'Not With Team']).any(axis=1)
boxscore_df = boxscore_df[~mask]
for col in ['FG','FGA','3P','3PA','FT','FTA','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','+/-']:
    boxscore_df[col] = pd.to_numeric(boxscore_df[col])

In [18]:
grouped = boxscore_df.groupby(['game_id', 'team_name'])

# First, aggregate the stats in your main dataframe
aggregated_df = grouped[['FG','FGA','3P','3PA','FT','FTA','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','+/-']].sum().reset_index()

# Calculate percentages for the team
aggregated_df['FG%'] = aggregated_df['FG'] / aggregated_df['FGA']
aggregated_df['FT%'] = aggregated_df['FT'] / aggregated_df['FTA']
aggregated_df['3P%'] = aggregated_df['3P'] / aggregated_df['3PA']

# Merge with the games_df to get the datetime, home_team, away_team, and is_regular values
aggregated_df = pd.merge(aggregated_df, 
                         games_df[['game_id', 'home_team', 'away_team', 'datetime', 'is_regular']], 
                         on='game_id', 
                         how='left')

# Create 'opponent' column
aggregated_df['opponent'] = np.where(aggregated_df['team_name'] == aggregated_df['home_team'], 
                                     aggregated_df['away_team'], 
                                     aggregated_df['home_team'])
# Create 'Home/Away' column
aggregated_df['home'] = np.where(aggregated_df['team_name'] == aggregated_df['home_team'], 
                                 'Home', 
                                 'Away')


# Drop the now redundant columns if you wish
aggregated_df = aggregated_df.drop(['home_team', 'away_team'], axis=1)

aggregated_df['win'] = aggregated_df.groupby('game_id')['PTS'].transform(lambda x: x == x.max())

aggregated_df.head()


,game_id,team_name,FG,FGA,3P,3PA,FT,FTA,ORB,DRB,...,PTS,+/-,FG%,FT%,3P%,datetime,is_regular,opponent,home,win
0,000079d5-37c3-4893-9f2c-d0776e80640c,New York Knicks,36.0,90.0,4.0,21.0,14.0,20.0,15.0,33.0,...,90.0,-20.0,0.400000,0.700000,0.190476,2014-02-05,1,Portland Trail Blazers,Home,False
1,000079d5-37c3-4893-9f2c-d0776e80640c,Portland Trail Blazers,30.0,79.0,8.0,21.0,26.0,33.0,12.0,37.0,...,94.0,20.0,0.379747,0.787879,0.380952,2014-02-05,1,New York Knicks,Away,True
2,00034685-1fcc-4465-ab33-473de0d6ba31,Houston Rockets,42.0,86.0,21.0,49.0,8.0,9.0,13.0,37.0,...,113.0,115.0,0.488372,0.888889,0.428571,2019-03-24,1,New Orleans Pelicans,Away,True
3,00034685-1fcc-4465-ab33-473de0d6ba31,New Orleans Pelicans,38.0,92.0,7.0,28.0,7.0,10.0,15.0,29.0,...,90.0,-115.0,0.413043,0.700000,0.250000,2019-03-24,1,Houston Rockets,Home,False
4,000702bc-3181-48fa-88ce-6e0fabc44917,Denver Nuggets,43.0,94.0,3.0,13.0,21.0,24.0,14.0,30.0,...,110.0,50.0,0.457447,0.875000,0.230769,2004-04-10,1,Portland Trail Blazers,Home,True


In [19]:
# Assuming the first entry for each game_id is the home team and the second is the away team
home_df = aggregated_df.iloc[::2].reset_index(drop=True)
away_df = aggregated_df.iloc[1::2].reset_index(drop=True)

# List of columns you want to prefix (excluding 'game_id' and other non-stat columns)
cols_to_prefix = ['FG','FGA','3P','3PA','FT','FTA','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','+/-', 'FG%', 'FT%', '3P%']

# Rename these columns in away_df to have the prefix 'Opp.'
away_df_prefixed = away_df[['game_id'] + cols_to_prefix].rename(columns={col: 'Opp.' + col for col in cols_to_prefix})

# Merge the home_df with the renamed away_df to get team stats and opponent stats in one row
combined_df1 = pd.merge(home_df, away_df_prefixed, on='game_id', how='left')

# Now, for the reverse: Rename the columns in home_df to have the prefix 'Opp.'
home_df_prefixed = home_df[['game_id'] + cols_to_prefix].rename(columns={col: 'Opp.' + col for col in cols_to_prefix})

# Merge the away_df with the renamed home_df
combined_df2 = pd.merge(away_df, home_df_prefixed, on='game_id', how='left')

# Concatenate the two dataframes to have records for both teams with their opponent stats
final_df = pd.concat([combined_df1, combined_df2]).sort_values(by='game_id').reset_index(drop=True)

final_df.columns

Index(['game_id', 'team_name', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-', 'FG%',
       'FT%', '3P%', 'datetime', 'is_regular', 'opponent', 'home', 'win',
       'Opp.FG', 'Opp.FGA', 'Opp.3P', 'Opp.3PA', 'Opp.FT', 'Opp.FTA',
       'Opp.ORB', 'Opp.DRB', 'Opp.TRB', 'Opp.AST', 'Opp.STL', 'Opp.BLK',
       'Opp.TOV', 'Opp.PF', 'Opp.PTS', 'Opp.+/-', 'Opp.FG%', 'Opp.FT%',
       'Opp.3P%'],
      dtype='object')

In [20]:
# Define the desired order of columns
desired_order = ['game_id', 'team_name', 'datetime', 'is_regular', 'opponent', 'home', 'win'] + [col for col in final_df.columns if col not in ['game_id', 'team_name', 'datetime', 'is_regular', 'opponent', 'home', 'win']]

# Re-order the columns in the DataFrame
final_df = final_df[desired_order]

final_df = final_df.sort_values(by=['game_id', 'home', 'datetime'])
final_df.to_csv('../data/src/boxscore_aggregates.csv', index=False)